In [1]:
import json
import os

import dspy

In [2]:
def load_math_as_examples(path="../datasets/MATH", split="train"):
    with open(os.path.join(path, split, "dataset.json")) as f:
        data = json.load(f)
    
    examples = []

    for question, answer in zip(data["question"], data["extracted_answers"]):
        example = dspy.Example(question=question, answer=answer).with_inputs("question")
        examples.append(example)

    return examples

In [3]:
HOST = "localhost"
# HOST = "babel-15-20"
PORT = 8000
API_KEY = "EMPTY"
API_BASE = f"http://{HOST}:{PORT}/v1"
MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"
# MODEL_NAME = "llm"

lm = dspy.LM("openai/"+MODEL_NAME, api_base=API_BASE, api_key=API_KEY, model_type='chat', cache=False)
dspy.configure(lm=lm)

In [5]:
dataset = load_math_as_examples(split="test")
len(dataset)

5000

In [75]:
class MathSignature(dspy.Signature):
    """Answer the math question."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="The final answer in latex format. Do not include the \\boxed{} symbol.")

class GenerateSignature(dspy.Signature):
    """Create similar math questions and answers as the ones in the input"""

    question = dspy.InputField()
    answer = dspy.InputField()
    generated_examples: list[dict[str, str]] = dspy.OutputField(desc="1 generated similar question and answer")

class MathManyShotSignature(dspy.Signature):
    """Answer the math question."""

    examples: list[dict[str, str]] = dspy.InputField(desc="List of examples with question and answer")
    # examples: str = dspy.InputField(desc="List of examples with question and answer")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="The final answer in latex format. Do not include the \\boxed{} symbol.")

In [56]:
augment = dspy.Predict(GenerateSignature, n=10)

In [57]:
response = augment(**dataset[0])
response.completions.generated_examples

[[{'question': 'A fair coin is flipped 4 times.  What is the probability that we get at most 2 heads?',
   'answer': '\\frac{17}{32}'}],
 [{'question': 'We roll a fair 6-sided die 4 times.  What is the probability that we get a 6 in at most 2 of the rolls?',
   'answer': '\\frac{225}{384}'}],
 [{'question': 'A fair 6-sided die is rolled 4 times.  What is the probability that we get a 6 in at most 1 of the rolls?',
   'answer': '\\frac{211}{240}'},
  {'question': 'We roll a fair 6-sided die 3 times.  What is the probability that we get a 6 in at most 1 of the rolls?',
   'answer': '\\frac{125}{216}'},
  {'question': 'A fair 6-sided die is rolled 6 times.  What is the probability that we get a 6 in at most 3 of the rolls?',
   'answer': '\\frac{341875}{2123368}'},
  {'question': 'We roll a fair 6-sided die 7 times.  What is the probability that we get a 6 in at most 3 of the rolls?',
   'answer': '\\frac{269350}{2985984}'}],
 [{'question': 'A fair 10-sided die is rolled 4 times. What is 

In [59]:
generated_examples = [ge[0] for ge in response.completions.generated_examples]
generated_examples

[{'question': 'A fair coin is flipped 4 times.  What is the probability that we get at most 2 heads?',
  'answer': '\\frac{17}{32}'},
 {'question': 'We roll a fair 6-sided die 4 times.  What is the probability that we get a 6 in at most 2 of the rolls?',
  'answer': '\\frac{225}{384}'},
 {'question': 'A fair 6-sided die is rolled 4 times.  What is the probability that we get a 6 in at most 1 of the rolls?',
  'answer': '\\frac{211}{240}'},
 {'question': 'A fair 10-sided die is rolled 4 times. What is the probability that we get a 10 in at most 1 of the rolls?',
  'answer': '\\frac{576}{625}'},
 {'question': 'We flip a fair coin 8 times. What is the probability that we get heads in at most 3 of the flips?',
  'answer': '\\frac{6435}{65536}'},
 {'question': 'We roll a fair 6-sided die 4 times.  What is the probability that we get a 6 in at most 1 of the rolls?',
  'answer': '\\frac{369}{1296}'},
 {'question': 'We roll a fair 4-sided die 5 times.  What is the probability that we get a 4 i

In [71]:
# Convert generated examples to a string
generated_examples_str = "\n\n".join([f"question: {ge['question']!r}\nanswer: {ge['answer']!r}" for ge in generated_examples])
print(generated_examples_str)

question: 'A fair coin is flipped 4 times.  What is the probability that we get at most 2 heads?'
answer: '\\frac{17}{32}'

question: 'We roll a fair 6-sided die 4 times.  What is the probability that we get a 6 in at most 2 of the rolls?'
answer: '\\frac{225}{384}'

question: 'A fair 6-sided die is rolled 4 times.  What is the probability that we get a 6 in at most 1 of the rolls?'
answer: '\\frac{211}{240}'

question: 'A fair 10-sided die is rolled 4 times. What is the probability that we get a 10 in at most 1 of the rolls?'
answer: '\\frac{576}{625}'

question: 'We flip a fair coin 8 times. What is the probability that we get heads in at most 3 of the flips?'
answer: '\\frac{6435}{65536}'

question: 'We roll a fair 6-sided die 4 times.  What is the probability that we get a 6 in at most 1 of the rolls?'
answer: '\\frac{369}{1296}'

question: 'We roll a fair 4-sided die 5 times.  What is the probability that we get a 4 in at most 2 of the rolls?'
answer: '29/32'

question: 'A fair co

In [76]:
predict = dspy.ChainOfThought(MathManyShotSignature)

In [77]:
response = predict(examples=generated_examples, **dataset[0].inputs())
# response = predict(examples=generated_examples_str, **dataset[0].inputs())
response

Prediction(
    reasoning='To solve this problem, we can use the binomial probability formula. The probability of getting a 6 in at most 2 of the rolls is equal to the probability of getting 0 sixes, plus the probability of getting 1 six, plus the probability of getting 2 sixes.\n\nThe probability of getting a 6 in a single roll is 1/6, and the probability of not getting a 6 is 5/6. We can use the binomial coefficient to calculate the number of ways to get 0, 1, or 2 sixes in 5 rolls.\n\nThe probability of getting 0 sixes is (5/6)^5, since we need to not get a 6 in all 5 rolls.\n\nThe probability of getting 1 six is 5C1 * (1/6)^1 * (5/6)^4, since we need to choose 1 roll out of 5 to be a 6, and the other 4 rolls need to not be 6s.\n\nThe probability of getting 2 sixes is 5C2 * (1/6)^2 * (5/6)^3, since we need to choose 2 rolls out of 5 to be 6s, and the other 3 rolls need to not be 6s.\n\nWe can add these probabilities together to get the final answer.',
    answer='\\\\frac{211}{243}'

In [78]:
dspy.inspect_history()





[2024-11-21T18:45:59.945643]

System message:

Your input fields are:
1. `examples` (list[dict[str, str]]): List of examples with question and answer
2. `question` (str)

Your output fields are:
1. `reasoning` (str)
2. `answer` (str): The final answer in latex format. Do not include the \boxed{} symbol.

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## examples ## ]]
{examples}

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Answer the math question.


User message:

[[ ## examples ## ]]
[{"question": "A fair coin is flipped 4 times.  What is the probability that we get at most 2 heads?", "answer": "\\frac{17}{32}"}, {"question": "We roll a fair 6-sided die 4 times.  What is the probability that we get a 6 in at most 2 of the rolls?", "answer": "\\frac{225}{384}"}, {"question": "A fair 6-sided die 